In [ ]:
import geohash2
import folium
import geopandas as gpd
from shapely.geometry import Point
from folium.plugins import MeasureControl, Fullscreen
import random
from haversine import haversine
from IPython.display import display
from shapely.geometry import Polygon
import psycopg2

# Load GeoJSON files for campus boundaries and different areas
CAMPUS_AREA_CATEGORY_LAYER = 'CAMPUS_AREA_CATEGORY_LAYER.geojson'
BUILDING_CODES_LAYER = 'AcademicBuildingCodesNew.geojson'
IIT_KGP_NEW_BOUNDARY = 'IIT_KGP_NEW_BOUNDARY.geojson'
IIT_KGP_LOCATIONS_POINT_LAYER = 'IIT_KGP_LOCATIONS_POINT.geojson'

IIT_KGP_NEW_BOUNDARY_gdf = gpd.read_file(IIT_KGP_NEW_BOUNDARY)
BUILDING_CODES_gdf = gpd.read_file(BUILDING_CODES_LAYER)
CAMPUS_AREA_CATEGORY_gdf = gpd.read_file(CAMPUS_AREA_CATEGORY_LAYER)
IIT_KGP_LOCATIONS_POINT_gdf = gpd.read_file(IIT_KGP_LOCATIONS_POINT_LAYER)

locationName = ""
locationDescription= ""
roomNo = ""
floorNo = None

# Connect to PostgreSQL
conn = psycopg2.connect(
    host='localhost',
    port='5433',
    user='postgres',
    password='postgres',
    database='Geohash'
)

# Create a cursor
cursor = conn.cursor()

# Create the geohash table if it doesn't exist
create_table_query = """
    CREATE TABLE IF NOT EXISTS geohash3 (
        geohashcode VARCHAR(15),
        customcode VARCHAR(15),
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        location_name VARCHAR(500),
        location_description VARCHAR(5000),
        RoomNo VARCHAR(100)
    );
"""
cursor.execute(create_table_query)
conn.commit()


# Extract the campus boundary polygon from the GeoDataFrame
campus_polygon = IIT_KGP_NEW_BOUNDARY_gdf.geometry.iloc[0]

# Calculate the centroid of the campus polygon
centroid = campus_polygon.centroid

# Create a folium map centered around the campus
campus_map = folium.Map(
    location=[centroid.y, centroid.x],
    zoom_start=15,
    height=600
)

# Add Esri satellite imagery as the basemap
esri_satellite_url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
folium.TileLayer(
    tiles=esri_satellite_url,
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
).add_to(campus_map)

# Generate and display geohash grids inside the campus polygon for precision level 8
grid_size = 150  # Number of points in each direction
bounds = campus_polygon.bounds  # Get the bounding box of the campus polygon

# Create a list to store geohash codes for precision level 8
geohash_codes = []
count=0

# Get only the Academic category from CAMPUS_AREA_CATEGORY_gdf
academic_polygon = CAMPUS_AREA_CATEGORY_gdf[CAMPUS_AREA_CATEGORY_gdf['Category'] == 'Academic'].geometry.unary_union

step_lat = (bounds[3] - bounds[1]) / grid_size
step_lon = (bounds[2] - bounds[0]) / grid_size

for i in range(grid_size):
    for j in range(grid_size):
        latitude = bounds[1] + i * step_lat
        longitude = bounds[0] + j * step_lon
        point = Point(longitude, latitude)

        # Check if the generated point is within the academic area
        if academic_polygon.contains(point):
            # Fetch building code from BUILDING_CODES_gdf
            code_df = BUILDING_CODES_gdf[BUILDING_CODES_gdf.geometry.contains(point)]
            if not code_df.empty:
                building_code = code_df.iloc[0]['CODE']
            else:
                building_code = "FZ"  # Non-Building if not in any building

            # Generate a random two-digit number
            random_number = str(random.randint(10, 99))
            original_geohash = geohash2.encode(latitude, longitude, precision=8)

            corners = [
            (latitude, longitude),
            (latitude + step_lat, longitude),
            (latitude + step_lat, longitude + step_lon),
            (latitude, longitude + step_lon),
                ]
            
            grid_polygon = geohash2.decode_exactly(original_geohash)
            grid_bounds = [grid_polygon[0], grid_polygon[1], grid_polygon[0] + grid_polygon[2], grid_polygon[1] + grid_polygon[3]]

            # Iterate over the points in the GeoDataFrame
            for index, row in IIT_KGP_LOCATIONS_POINT_gdf.iterrows():
                point = row.geometry
    
                # Decode the original geohash
                grid_polygon = geohash2.decode_exactly(original_geohash)
    
                # Calculate the size of the buffer (50% of the grid's dimensions)
                buffer_lat = 0.25 * grid_polygon[2]
                buffer_lon = 0.25 * grid_polygon[3]
    
                # Define the grid bounds with the buffer
                grid_bounds = [grid_polygon[0] - buffer_lat, grid_polygon[1] - buffer_lon, grid_polygon[0] + grid_polygon[2] + buffer_lat, grid_polygon[1] + grid_polygon[3] + buffer_lon]
    
                # Check if the point is within the buffered grid
                if grid_bounds[0] <= point.y <= grid_bounds[2] and grid_bounds[1] <= point.x <= grid_bounds[3]:
                
                    locationName = row.LocationName
                    locationDescription = row.LocationDescription
                    floorNo = row.FloorNo
                    roomNo = row.RoomNo
                    
                    if str(roomNo).isnumeric():
                        print("roomNo", roomNo)
                        custom_geohash = f'A{building_code[:2]}{row.FloorNo}{roomNo}'  # Use random.randint to generate a random number
                        print("custom_geohash", custom_geohash)
                    else:
                        custom_geohash = f'A{building_code[:2]}{row.FloorNo}{random.randint(10, 99)}'  
                        
                    insert_query = """
                    INSERT INTO geohash3 (geohashcode, customcode, latitude, longitude, location_name, location_description, RoomNo)
                    VALUES (%s, %s, %s, %s, %s, %s, %s);
                    """
                    cursor.execute(insert_query, (geohash2.encode(latitude, longitude, precision=8),
                                                  custom_geohash, latitude, longitude, locationName, locationDescription, roomNo))
                    conn.commit()
                    count+=1
                
                    
            if floorNo == None:
                custom_geohash = f'A{building_code[:2]}G{random.randint(10, 99)}'  # Use random.randint to generate a random number
                insert_query = """
                INSERT INTO geohash3 (geohashcode, customcode, latitude, longitude, location_name, location_description, RoomNo)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """
                cursor.execute(insert_query, (geohash2.encode(latitude, longitude, precision=8),
                                                  custom_geohash, latitude, longitude, locationName, locationDescription, roomNo))
                conn.commit()
                
            geohash_codes.append(custom_geohash)
            
            # Calculate the distance from the centroid to the current point
            distance_to_centroid = haversine((centroid.y, centroid.x), (latitude, longitude))

            # Create a popup with the custom geohash code and distance to centroid
            popup_text = f'Custom Geohash: {custom_geohash} <br>Custom Code: {custom_geohash}<br>Distance to Centroid: {distance_to_centroid:.2f} km <br>Location Name: {locationName} <br>Location Description: {locationDescription} '
            popup = folium.Popup(popup_text, max_width=400)

            # Create a polygon using the coordinates
            folium.Polygon(
                locations=corners,
                popup=popup,
                color='red',
                fill=True,
                fill_color='#000000',
                fill_opacity=0.3,
                weight=0.5,
                ).add_to(campus_map)
            
            
# Close the cursor and connection
cursor.close()
conn.close()

# Add the MeasureControl to the map
campus_map.add_child(MeasureControl(primary_length_unit='meters', primary_area_unit='sqmeters'))

Fullscreen().add_to(campus_map)
geohashcode11 = geohash2.encode(centroid.y, centroid.x, precision=8)

folium.Marker([centroid.y, centroid.x], tooltip="Centroid: " + geohashcode11).add_to(campus_map)

# Display the map with Esri satellite imagery as the basemap, custom geohash grids, and the measurement tool
display(campus_map)

# Save the map to an HTML file
output_html_file = "campus_map.html"
campus_map.save(output_html_file)
print("count", count)